<a href="https://colab.research.google.com/github/tjonty/Assessment-Product-Team-Sync/blob/main/Product_Team_Sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


MARKDOWN_NOTES = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
MENTION_RE = re.compile(r"@[\w-]+")

@dataclass
class LineItem:
    text: str
    kind: str  # "h1", "h2", "h3", "para", "bullet", "checkbox", "hr", "footer"
    level: int = 0  # nesting level for bullets/checkbox
    raw: str = ""


def _count_leading_spaces(s: str) -> int:
    return len(s) - len(s.lstrip(" "))


def parse_markdown(md: str) -> List[LineItem]:
    items: List[LineItem] = []
    lines = md.splitlines()

    hr_seen = False
    for raw in lines:
        line = raw.rstrip("\n")

        if not line.strip():
            # keep blank line as paragraph spacing
            items.append(LineItem(text="", kind="para", raw=raw))
            continue

        if line.strip() == "---":
            items.append(LineItem(text="", kind="hr", raw=raw))
            hr_seen = True
            continue

        # footer: anything after the HR that isn't empty is footer
        if hr_seen:
            items.append(LineItem(text=line.strip(), kind="footer", raw=raw))
            continue

        if line.startswith("# "):
            items.append(LineItem(text=line[2:].strip(), kind="h1", raw=raw))
            continue
        if line.startswith("## "):
            items.append(LineItem(text=line[3:].strip(), kind="h2", raw=raw))
            continue
        if line.startswith("### "):
            items.append(LineItem(text=line[4:].strip(), kind="h3", raw=raw))
            continue

        # checkbox: - [ ] something  (also handle * [ ] just in case)
        m = re.match(r"^(\s*)[-*]\s+\[\s\]\s+(.*)$", line)
        if m:
            indent_spaces = len(m.group(1))
            level = max(0, indent_spaces // 2)  # 2 spaces per nesting level is common
            items.append(LineItem(text=m.group(2).strip(), kind="checkbox", level=level, raw=raw))
            continue

        # bullets: "-", "*" with indentation
        m = re.match(r"^(\s*)[-*]\s+(.*)$", line)
        if m:
            indent_spaces = len(m.group(1))
            level = max(0, indent_spaces // 2)
            items.append(LineItem(text=m.group(2).strip(), kind="bullet", level=level, raw=raw))
            continue

        # fallback: normal paragraph
        items.append(LineItem(text=line.strip(), kind="para", raw=raw))

    return items

In [ ]:
@dataclass
class Range:
    start: int
    end: int


@dataclass
class StyledMention:
    start: int
    end: int


@dataclass
class ParagraphBlock:
    rng: Range
    kind: str
    level: int


def create_doc(service, title: str) -> str:
    doc = service.documents().create(body={"title": title}).execute()
    return doc["documentId"]


def build_requests(items: List[LineItem]) -> Tuple[List[dict], List[ParagraphBlock], List[StyledMention], List[Range]]:
    """
    We insert all text first, then apply styles/bullets via batchUpdate ranges.
    Returns:
      - requests: base requests (insertText)
      - paragraphs: ranges that map to paragraph-level styling
      - mentions: ranges for @mention styling
      - footer_ranges: ranges for footer styling
    """
    requests: List[dict] = []
    paragraphs: List[ParagraphBlock] = []
    mentions: List[StyledMention] = []
    footer_ranges: List[Range] = []

    # New docs start with a single newline. Inserting at index=1 appends content at the start.
    cursor = 1

    for it in items:
        # For HR, we just add a blank line + maybe a divider-like text
        if it.kind == "hr":
            text = "\n"
            requests.append({"insertText": {"location": {"index": cursor}, "text": text}})
            cursor += len(text)
            continue

        # Decide the text we insert for this line
        line_text = it.text

        # Keep spacing: blank para -> just newline
        if it.kind == "para" and line_text == "":
            text = "\n"
            requests.append({"insertText": {"location": {"index": cursor}, "text": text}})
            cursor += len(text)
            continue

        # normal line ends with newline
        text = f"{line_text}\n"
        start = cursor
        end = cursor + len(text)

        requests.append({"insertText": {"location": {"index": cursor}, "text": text}})
        paragraphs.append(ParagraphBlock(rng=Range(start, end), kind=it.kind, level=it.level))

        # Track mention ranges inside this inserted line
        for m in MENTION_RE.finditer(line_text):
            ms = start + m.start()
            me = start + m.end()
            mentions.append(StyledMention(start=ms, end=me))

        # Track footer ranges
        if it.kind == "footer":
            footer_ranges.append(Range(start, end))

        cursor = end

    return requests, paragraphs, mentions, footer_ranges


def paragraph_style_request(rng: Range, named_style: str) -> dict:
    return {
        "updateParagraphStyle": {
            "range": {"startIndex": rng.start, "endIndex": rng.end},
            "paragraphStyle": {"namedStyleType": named_style},
            "fields": "namedStyleType",
        }
    }


def indent_paragraph_request(rng: Range, level: int) -> dict:
    # Keep indentation modest so it looks like a normal doc.
    # Google Docs uses points. 18pt per level is a decent default.
    indent_pts = 18.0 * level
    return {
        "updateParagraphStyle": {
            "range": {"startIndex": rng.start, "endIndex": rng.end},
            "paragraphStyle": {
                "indentStart": {"magnitude": indent_pts, "unit": "PT"},
                "indentFirstLine": {"magnitude": 0.0, "unit": "PT"},
            },
            "fields": "indentStart,indentFirstLine",
        }
    }


def bullets_request(rng: Range, preset: str) -> dict:
    return {
        "createParagraphBullets": {
            "range": {"startIndex": rng.start, "endIndex": rng.end},
            "bulletPreset": preset,
        }
    }


def mention_text_style_request(start: int, end: int) -> dict:
    # Bold + blue-ish text, common mention styling
    return {
        "updateTextStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "textStyle": {
                "bold": True,
                "foregroundColor": {
                    "color": {
                        "rgbColor": {"red": 0.12, "green": 0.35, "blue": 0.85}
                    }
                },
            },
            "fields": "bold,foregroundColor",
        }
    }


def footer_text_style_request(rng: Range) -> dict:
    return {
        "updateTextStyle": {
            "range": {"startIndex": rng.start, "endIndex": rng.end},
            "textStyle": {
                "italic": True,
                "fontSize": {"magnitude": 10, "unit": "PT"},
                "foregroundColor": {
                    "color": {"rgbColor": {"red": 0.45, "green": 0.45, "blue": 0.45}}
                },
            },
            "fields": "italic,fontSize,foregroundColor",
        }
    }


def convert_markdown_to_gdoc(md: str, doc_title: str = "Product Team Sync") -> str:
    try:
        auth.authenticate_user()
        docs_service = build("docs", "v1")

        doc_id = create_doc(docs_service, doc_title)

        items = parse_markdown(md)
        insert_reqs, paragraphs, mentions, footer_ranges = build_requests(items)

        style_reqs: List[dict] = []

        # Paragraph styles + bullets/checkboxes
        for p in paragraphs:
            if p.kind == "h1":
                style_reqs.append(paragraph_style_request(p.rng, "HEADING_1"))
            elif p.kind == "h2":
                style_reqs.append(paragraph_style_request(p.rng, "HEADING_2"))
            elif p.kind == "h3":
                style_reqs.append(paragraph_style_request(p.rng, "HEADING_3"))
            elif p.kind in ("bullet", "checkbox"):
                # Bullets/checkboxes apply to paragraph ranges including newline.
                preset = "BULLET_DISC_CIRCLE_SQUARE" if p.kind == "bullet" else "BULLET_CHECKBOX"
                style_reqs.append(bullets_request(p.rng, preset))
                if p.level > 0:
                    style_reqs.append(indent_paragraph_request(p.rng, p.level))

        # Mention styling
        for m in mentions:
            style_reqs.append(mention_text_style_request(m.start, m.end))

        # Footer styling
        for fr in footer_ranges:
            style_reqs.append(footer_text_style_request(fr))

        # Run requests: insert first, then styles
        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={"requests": insert_reqs}
        ).execute()

        if style_reqs:
            docs_service.documents().batchUpdate(
                documentId=doc_id,
                body={"requests": style_reqs}
            ).execute()

        return doc_id

    except HttpError as e:
        # API error (auth, permissions, bad requests, etc.)
        raise RuntimeError(f"Google Docs API error: {e}") from e
    except Exception as e:
        raise RuntimeError(f"Conversion failed: {e}") from e



In [ ]:
doc_id = convert_markdown_to_gdoc(MARKDOWN_NOTES, doc_title="Product Team Sync")
print("Created Google Doc:")
print(f"https://docs.google.com/document/d/{doc_id}/edit")

Created Google Doc:
https://docs.google.com/document/d/1OdikunaGmreDIP4SVEGxHcYCqcXDwab_skmQPvVh3es/edit
